In [292]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch 
import torch.nn as nn
import uuid
import pickle 
from joblib import dump, load

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from mlflow.data.pandas_dataset import PandasDataset

import mlflow
from mlflow.entities import Metric
from mlflow.tracking import MlflowClient
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')


In [293]:
CONFIG = {
    'data_directory':r'/home/epsilonoptima/repos/cuda prototype/',
    'partial_filename':'spx_data',
    'reverse_data_order':True,
    'set_date_index':False,
    'mlflow_uri':"sqlite:////home/epsilonoptima/mlflow/mlruns.db",
    'artifact_directory': '////home/epsilonoptima/mlflow/artifacts',
    'temp_directory':'////home/epsilonoptima/mlflow/temp/cuda0',
    'experiment_name':"SPX_1D_BC_Proto_10",
    'column_filter':1,
    'datasource_name':'yfinance'
         }

In [294]:
# Set up ML Flow
mlflow.set_tracking_uri(CONFIG['mlflow_uri'])
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
curr_experiment = mlflow.set_experiment(CONFIG['experiment_name'])

2023/12/19 15:21:50 INFO mlflow.tracking.fluent: Experiment with name 'SPX_1D_BC_Proto_10' does not exist. Creating a new experiment.


In [295]:
def find_and_load_csv_into_df():
    for dirname, _, filenames in os.walk(CONFIG['data_directory']):
        for filename in filenames:
            if CONFIG['partial_filename'] in filename:
                return pd.read_csv(os.path.join(dirname, filename), index_col=0)


In [296]:
def create_df():
    df = find_and_load_csv_into_df()

    df.drop(columns=['Adj Close'], errors='ignore', inplace=True)
    df = df.iloc[::-1]

    if CONFIG['set_date_index']:
        df['Date'] = pd.to_datetime(df['Date'])
        # set this to true if not using the new YF dataset
        df.set_index('Date', inplace=True)
        
    if CONFIG['reverse_data_order']:
        df = df.reindex(index=df.index[::-1])
        df.index = pd.to_datetime(df.index).date

    col = df.pop('Close')
    df.insert(df.shape[1], 'Close', col)

    df['Tomorrow_Close'] = df['Close'].shift(-1)
    df_with_final_row = df.copy()
    df.dropna(inplace=True)

    n = df.shape[0]

    # Filter rows to just have OHLCV and TNX
    if CONFIG['column_filter'] == 0:
        df = df.loc[:,['Open', 'High', 'Low', 'Volume', 'Close_^TNX', 'Close','Tomorrow_Close']]
    elif CONFIG['column_filter'] == 1:
        df = df.loc[:,[col for col in df.columns if 'Close' in col]]

    # Turn tomorrow close into a binary classification

    df['Tomorrow_Close'] = np.where(df['Tomorrow_Close'] >= df['Close'], 1.0, 0.0)
    
    return df, df_with_final_row


In [297]:
df, df_with_final_row = create_df()
dataset: PandasDataset = mlflow.data.from_pandas(df, source=CONFIG['datasource_name'])

In [298]:
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
print(f"Device : {dev}")
device = torch.device(dev) 

Device : cuda:0


In [299]:
# Credit to Charlie Oneil for this dataset creator

# split a multivariate sequence past, future samples (X and y)
def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out, n_step_gap = 1):
    X, y = list(), list() # instantiate X and y
    input_rows = input_sequences.shape[0]
    for i in range(input_rows):
        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        # check if we are beyond the dataset
        if out_end_ix > len(input_sequences): break
        # gather input and output of the pattern
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix-1:out_end_ix, -1]
        X.append(seq_x), y.append(seq_y)
    return np.array(X), np.array(y)

In [300]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, fc_neurons, device):
        super().__init__()
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        self.device = device
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size, fc_neurons) # fully connected 
        self.fc_2 = nn.Linear(fc_neurons, 1) # fully connected last layer
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        
        hn = hn[-1]
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        
        
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        out = self.sigmoid(out)
        return out

In [301]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train,
                  X_test = None, y_test = None, early_stopper = None):
    train_metrics = []
    for epoch in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train) # forward pass
        optimiser.zero_grad() # calculate the gradient, manually setting to 0
        # obtain the loss function
        loss = loss_fn(outputs, y_train)
        loss.backward() # calculates the loss of the loss function
        
        # obtain the accuracy
        acc = (outputs.round() == y_train).float().mean()
        
        # consider gradient clipping / normalization
        # torch.nn.utils.clip_grad_norm(lstm.parameters(), max_norm=1)
        
        optimiser.step() # improve from loss, i.e backprop
                
        # test loss
        if X_test is not None and y_test is not None:
            lstm.eval()
            test_preds = lstm(X_test)
            test_loss = loss_fn(test_preds, y_test)
            test_acc = (test_preds.round() == y_test).float().mean()
            train_metrics.append({
                'epoch':epoch+1, 
                'train_loss':loss.item(),
                'train_acc':acc.item(), 
                'test_loss':test_loss.item(), 
                'test_acc':test_acc.item()
            })
            
            if early_stopper is not None:
                if early_stopper.early_stop(test_loss.item()):
                    return epoch+1, train_metrics
        else:
            train_metrics.append({
                'epoch':epoch+1, 
                'train_loss':loss.item(),
                'train_acc':acc.item()
            })
    
    return n_epochs, train_metrics

In [302]:
class EarlyStopper:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


In [303]:
def get_start_end_idx(test_set_len=252):
    return df_with_final_row.shape[0]-test_set_len, df_with_final_row.shape[0]

def make_tensors(ss, data, seq_len):
    X = data.iloc[:,:-1].values
    y = data.iloc[:,-1:].values

    X_trans = ss.fit_transform(X)

    X_ss, y_seq = split_sequences(X_trans, y, seq_len, 1)

    # To simulate real market inference conditions, we leave a 1 day gap between training and inference
    X_train = X_ss[:-2]
    y_train = y_seq[:-2]

    X_test = X_ss[-1:]
    y_test = y_seq[-1:]

    X_train_tensors = torch.Tensor(X_train).to(device)
    y_train_tensors = torch.Tensor(y_train).to(device)

    # Tensor dimensions (batch size, sequence, features)
    X_train_tensors = torch.reshape(X_train_tensors,   
                                      (X_train_tensors.shape[0], seq_len, 
                                       X_train_tensors.shape[2]))

    X_test_tensors = torch.Tensor(X_test).to(device)
    y_test_tensors = torch.Tensor(y_test).to(device)

    X_test_tensors = torch.reshape(X_test_tensors,   
                                      (X_test_tensors.shape[0], seq_len, 
                                       X_test_tensors.shape[2]))
    
    return X_train_tensors, y_train_tensors, X_test_tensors, y_test_tensors

def make_cross_val_df(idx,sw):
    if sw is None:
        return df.iloc[:idx]
    else:
        return df.iloc[idx-sw:idx]
        

In [304]:
def determine_epochs(**kwargs):
    start_idx, end_idx = get_start_end_idx()
    cv_epochs = []

    for i in range(start_idx, end_idx, 1):
        df_cv = make_cross_val_df(i, kwargs['sliding_window'])
        input_size = df_cv.shape[1]-1
        
        ss = StandardScaler()
        X_train_tensors, y_train_tensors, X_test_tensors, y_test_tensors = make_tensors(ss, df_cv, kwargs['seq_len'])
        
        train_val_split = int(X_train_tensors.shape[0]-1)

        lstm = LSTM(input_size, kwargs['hidden_size'], kwargs['num_layers'], kwargs["fc_neurons"], device).to(device)
        loss_fn = torch.nn.BCELoss()    # mean-squared error for regression
        # Foreach set to false because of PyTorch bug.
        optimiser = torch.optim.Adam(lstm.parameters(), lr=kwargs['learning_rate'])
        early_stopper = EarlyStopper()


        n_completed_epochs, train_metric = training_loop(
            n_epochs=kwargs['n_epochs'],
            lstm=lstm,
            optimiser=optimiser,
            loss_fn=loss_fn,
            X_train=X_train_tensors[:train_val_split],
            y_train=y_train_tensors[:train_val_split], 
            X_test=X_train_tensors[train_val_split:], 
            y_test=y_train_tensors[train_val_split:],
            early_stopper=early_stopper
        )
        cv_epochs.append(n_completed_epochs)
        
    return int(sum(cv_epochs)/len(cv_epochs))

def add_model_to_artifacts(lstm, model_name, artifacts):
    # Save the model and make it an artifact
    model_scripted = torch.jit.script(lstm) # Export to TorchScript
    model_filename = f"temp/last_feature_date_{model_name}.pt"
    model_scripted.save(model_filename) # Save
    artifacts.append(model_filename)
    
def log_result_mlflow(params, metrics, df_metrics, lstm_acc, artifacts, run):
    mlflow_client = MlflowClient()
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    epoch_metrics = []
    cols = df_metrics.columns
    for epoch in df_metrics.index:
        current_epoch_metrics = {col:df_metrics.loc[epoch,col] for col in df_metrics.columns}
        for k,v in current_epoch_metrics.items():
            metric = Metric(key=k,
                            value=v,
                            timestamp=0,
                            step=epoch)
            epoch_metrics.append(metric)
#             mlflow.log_metrics(current_epoch_metrics, step=epoch)
    mlflow_client.log_batch(run_id = run.info.run_id, metrics=epoch_metrics)

    # Log an instance of the trained model for later use
    for key, local_path in artifacts.items():
        mlflow.log_artifact(local_path)

    mlflow.log_input(dataset, context="training")

In [311]:
# create class
class PyFuncPyTorchLSTM(mlflow.pyfunc.PythonModel):          
    def predict(self, context, X, y=None):
        print('predict')
        sl = self.parameters['seq_len']
        print('loaded sequence length')
        print(X)
        X = X[0].iloc[-sl:]
        print(f'loaded data : {X.shape}')
        # We need to rescale the new data using standard scaler
        
        X_ss = self.ss.transform(X)
        print('Transformed X')
        # We need to used scaled data and make prediction

        X_tensors = torch.unsqueeze(torch.Tensor(X_ss),0)
        print('Created Tensor')

        self.model.device = torch.device("cpu") 
        self.model.to(self.model.device)
        self.model.eval()
        print('Turned on model eval')
        pred = self.model(X_tensors)
        print('obtained prediction')
        output = pred.detach().cpu().numpy()
        print('detatched prediction')
        # 1 is a gain prediction
        # 0 is a loss prediction
        return output
    
    def load_context(self, context):
        # Standard Scaler artifact
        self.ss = load(context.artifacts['ss'])
        # Pytorch Model artifact
        self.model = torch.jit.load(context.artifacts['mdl'])

        # Model parameters artifact
        with open(context.artifacts['params'], 'rb') as f:
            self.parameters = pickle.load(f)

        print('loaded contexts')

In [312]:

def run_trial(**kwargs):
    start_idx, end_idx = get_start_end_idx()

    df_plot = pd.DataFrame()
    train_metrics = []
    artifacts = {}
    start_time = time.time()

    try:

        # Initiate the MLflow run context
        with mlflow.start_run(run_name = str(kwargs)) as run:
            n_epochs = determine_epochs(**kwargs)
            
            for i in range(start_idx, end_idx, 1):
                # print(f"Processing : {df_with_final_row.index[i]}")

                df_cv = make_cross_val_df(i, kwargs['sliding_window'])
                input_size = df_cv.shape[1]-1

                ss = StandardScaler()
                X_train_tensors, y_train_tensors, X_test_tensors, y_test_tensors = make_tensors(ss, df_cv, kwargs['seq_len'])
                
                
                lstm = LSTM(input_size, kwargs['hidden_size'], kwargs['num_layers'], kwargs["fc_neurons"], device).to(device)
                loss_fn = torch.nn.BCELoss()    # mean-squared error for regression
                # Foreach set to false because of PyTorch bug.
                optimiser = torch.optim.Adam(lstm.parameters(), lr=kwargs['learning_rate'])


                n_epochs, train_metric = training_loop(
                    n_epochs=n_epochs,
                    lstm=lstm,
                    optimiser=optimiser,
                    loss_fn=loss_fn,
                    X_train=X_train_tensors,
                    y_train=y_train_tensors, 
                    X_test=X_test_tensors, 
                    y_test=y_test_tensors
                )

                train_metrics = [*train_metrics, *train_metric]
                
                if i == end_idx - 1:
                    # We serialize our final standard scaler for inference
                    os.makedirs(CONFIG['artifact_directory']+f'/{run.info.run_id}/', exist_ok=True)
                    dump(ss, CONFIG['artifact_directory']+f'/{run.info.run_id}/standard_scalar.pkl')
                    artifacts['ss']=CONFIG['artifact_directory']+f'/{run.info.run_id}/standard_scalar.pkl'
                    
                    # Save the model and make it an artifact
                    model_scripted = torch.jit.script(lstm) # Export to TorchScript
                    model_filename = CONFIG['artifact_directory']+f'/{run.info.run_id}/model.pt'
                    model_scripted.save(model_filename) # Save
                    artifacts['mdl']=model_filename

                    # Save params as a dictionary in a file
                    param_dict_filename = CONFIG['artifact_directory']+f'/{run.info.run_id}/param_dict.pkl'
                    with open(param_dict_filename, 'wb') as f:
                        pickle.dump(kwargs, f)
                    artifacts['params']=param_dict_filename

                data_predict = lstm(X_test_tensors).cpu().data.numpy() # numpy conversion
                dataY_actual = np.array(df_cv.iloc[:,-1:].values[-1:])
                df_plot = pd.concat([df_plot, pd.DataFrame([
                    {'Actual Data':dataY_actual[0][0], 'Predicted Data':data_predict[0][0],'Date':df_with_final_row.index[i]}
                ])])

            
            end_time = time.time()

            df_metrics = pd.DataFrame(train_metrics)
            df_metrics = df_metrics.groupby('epoch').mean()

            df_plot['Predicted Data'] = df_plot['Predicted Data'].round()
            lstm_acc = (df_plot['Actual Data']==df_plot['Predicted Data']).mean()
            # Assemble the metrics we're going to write into a collection
            metrics = {"Accuracy": lstm_acc, "Elapsed" : round(end_time - start_time,1)}
            print(f"Overall accuracy of {str(kwargs)} : {lstm_acc}")
            
            kwargs['model_path'] = CONFIG['artifact_directory']+f'/{run.info.run_id}/model'
            mlflow.pyfunc.save_model(path=kwargs['model_path'], python_model=PyFuncPyTorchLSTM(), artifacts=artifacts)

            log_result_mlflow(kwargs, metrics, df_metrics, lstm_acc, artifacts, run)
    
    except Exception as e:
        print("Error aborting run. See errors after completion")
        return {f"{kwargs}":e}



In [313]:
sliding_window_lengths = [None, 75, 100, 150, 200, 250, 500, 750]
sequence_lengths = [3, 5, 7, 10, 15, 20, 25, 35, 50, 65, 80, 100, 125]
hidden_sizes = [1, 2, 3, 6, 16]
fc_neurons = [16,128,1024]
layers = [1,2,3]
num_replicates = 30

errors = []

for r in range(num_replicates):
    for sw in sliding_window_lengths[1:]:
        for seq in sequence_lengths:
            for hidden_size in hidden_sizes:
                for fc_neuron in fc_neurons:
                    for num_layers in layers:

                        if sw is not None and seq + 2 >= sw:
                            continue

                        params = {
                            "seq_len": seq,
                            "hidden_size": hidden_size,
                            "num_layers": num_layers,
                            "fc_neurons": fc_neuron,
                            "n_epochs": 200,
                            "learning_rate" : 0.001,
                            "sliding_window" : sw
                        }

                        trial = run_trial(**params)
                        if type(trial) is dict:
                            errors.append(trial)
                            

                    
for err in errors:
    print(err[0])
    print(err[1])
    print()

Overall accuracy of {'seq_len': 3, 'hidden_size': 1, 'num_layers': 1, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 75} : 0.47619047619047616


Overall accuracy of {'seq_len': 3, 'hidden_size': 1, 'num_layers': 2, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 75} : 0.46825396825396826


Overall accuracy of {'seq_len': 3, 'hidden_size': 1, 'num_layers': 3, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 75} : 0.47619047619047616


Overall accuracy of {'seq_len': 3, 'hidden_size': 1, 'num_layers': 1, 'fc_neurons': 128, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 75} : 0.46825396825396826


Overall accuracy of {'seq_len': 3, 'hidden_size': 1, 'num_layers': 2, 'fc_neurons': 128, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 75} : 0.4880952380952381


...

Overall accuracy of {'seq_len': 25, 'hidden_size': 2, 'num_layers': 1, 'fc_neurons': 1024, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 750} : 0.5198412698412699


Overall accuracy of {'seq_len': 25, 'hidden_size': 2, 'num_layers': 2, 'fc_neurons': 1024, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 750} : 0.5357142857142857


Overall accuracy of {'seq_len': 25, 'hidden_size': 2, 'num_layers': 3, 'fc_neurons': 1024, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 750} : 0.49206349206349204


Overall accuracy of {'seq_len': 25, 'hidden_size': 3, 'num_layers': 1, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': 750} : 0.42857142857142855


KeyboardInterrupt: 

In [126]:
errors

[{"{'seq_len': 3, 'hidden_size': 1, 'num_layers': 1, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': None}": IndexError('tuple index out of range')},
 {"{'seq_len': 3, 'hidden_size': 1, 'num_layers': 2, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': None}": IndexError('tuple index out of range')},
 {"{'seq_len': 3, 'hidden_size': 1, 'num_layers': 3, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': None}": IndexError('tuple index out of range')},
 {"{'seq_len': 3, 'hidden_size': 1, 'num_layers': 1, 'fc_neurons': 128, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': None}": IndexError('tuple index out of range')},
 {"{'seq_len': 3, 'hidden_size': 1, 'num_layers': 2, 'fc_neurons': 128, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': None}": IndexError('tuple index out of range')},
 {"{'seq_len': 3, 'hidden_size': 1, 'num_layers': 3, 'fc_neurons': 128, 'n_epochs': 200, 'learning_rate

In [122]:

artifacts={'ss': '////home/epsilonoptima/mlflow/artifacts/0ec8c7b1d07b4deebafcd8be6fb5cecf/standard_scalar.pkl',
 'mdl': '////home/epsilonoptima/mlflow/artifacts/0ec8c7b1d07b4deebafcd8be6fb5cecf/model.pt'}
model_path = '////home/epsilonoptima/mlflow/artifacts/0ec8c7b1d07b4deebafcd8be6fb5cecf/model'
mc = {'seq_len': 3}

# create class
class PyFuncPyTorchLSTM(mlflow.pyfunc.PythonModel):          
    def predict(self, context, X, y=None):
        # We need to rescale the new data using standard scaler
        print('predict')

        # We need to used scaled data and make prediction
        # 1 is a gain prediction
        # 0 is a loss prediction
        return [1]
    
    def load_context(self, context):
        # Standard Scaler artifact
        self.ss = load(context.artifacts['ss'])
        # Pytorch Model artifact
        self.model = torch.jit.load(context.artifacts['mdl'])

        # Model parameters artifact
        with open(context.artifacts['params'], 'rb') as f:
            self.params = pickle.load(f)

        print('loaded contexts')




# create instance of model, return weights dict 
m = PyFuncPyTorchLSTM()
d = m.fit()

# create model input for predict function as tuple
model_input = ([df, d])

# save and re-load from ML flow
mlflow.pyfunc.save_model(path=model_path, python_model=m, artifacts=artifacts)
m2 = mlflow.pyfunc.load_model(model_path)
m2.predict(model_input)

fit


2023/12/19 07:28:13 WARNING mlflow.pyfunc: Argument(s) seq_len were ignored since the model's ``pyfunc`` flavor doesn't accept model configuration. Use ``model_config`` when logging the model to allow it.


loaded contexts
any model_config?
None
predict


,col1,col2,pred
0,1,3,2
1,2,4,4


In [123]:



with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(params, f)
        
with open('saved_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

print(loaded_dict)

{'seq_len': 3, 'hidden_size': 1, 'num_layers': 2, 'fc_neurons': 16, 'n_epochs': 200, 'learning_rate': 0.001, 'sliding_window': None}


In [39]:
class PyFuncPyTorchLSTM(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input, params=None):
        # define model loading method

        # Need to accept a dataframe, create sequences (sequence length and data window need to be known)


        return [0]

    def load_context(self, context):
        # define model loading method

        # need to load standard scalar into an object from an artifact path

        # need to load the pytorch model into an object from an artifact path 

        


NameError: name 'lstm' is not defined

In [101]:
artifacts

{'ss': '////home/epsilonoptima/mlflow/artifacts/0ec8c7b1d07b4deebafcd8be6fb5cecf/standard_scalar.pkl',
 'mdl': '////home/epsilonoptima/mlflow/artifacts/0ec8c7b1d07b4deebafcd8be6fb5cecf/model.pt'}